Use pipelines as this https://github.com/wassname/representation-engineering/blob/random_comments_ignore/examples/honesty/honesty.ipynb



In [1]:
# import your package
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

import os
from pathlib import Path
from tqdm.auto import tqdm
from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")

from typing import Optional, List, Dict, Union, Tuple, Callable, Iterable


/media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

import transformers
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from src.repe import repe_pipeline_registry
repe_pipeline_registry()

from src.models.load import load_model
from src.extraction.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset

# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
# from sklearn.preprocessing import RobustScaler


In [3]:
# model_name_or_path = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ"
# model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
model_name_or_path = "TheBloke/WizardCoder-Python-13B-V1.0-GPTQ"

batch_size = 2

cfg = ExtractConfig(max_examples=(100, 100), model=model_name_or_path, max_length=666)
print(cfg)

model, tokenizer = load_model(model_name_or_path)


ExtractConfig(datasets=('amazon_polarity', 'super_glue:boolq', 'glue:qnli', 'imdb'), model='TheBloke/WizardCoder-Python-13B-V1.0-GPTQ', data_dirs=(), max_examples=(100, 100), num_shots=1, num_variants=-1, layers=(), seed=42, template_path=None, max_length=666)


2023-10-25 13:32:38.035 | INFO     | src.models.load:verbose_change_param:18 - changing pad_token_id from 32000 to 0
2023-10-25T13:32:38.035153+0800 INFO changing pad_token_id from 32000 to 0
2023-10-25 13:32:38.036 | INFO     | src.models.load:verbose_change_param:18 - changing padding_side from right to left
2023-10-25T13:32:38.036448+0800 INFO changing padding_side from right to left
2023-10-25 13:32:38.037 | INFO     | src.models.load:verbose_change_param:18 - changing truncation_side from right to left
2023-10-25T13:32:38.037328+0800 INFO changing truncation_side from right to left


In [4]:


# hidden_layers = list(range(-1, -model.config.num_hidden_layers, -1))
# hidden_layers = [f"model.layers.{i}" for i in range(8, model.config.num_hidden_layers, 3)]
# hidden_layers = list(range(8, model.config.num_hidden_layers, 3))
# hidden_layers



# rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)
# rep_reading_pipeline
# hidden_layers


In [5]:
# # cache busting for the transformers map and ds steps
!rm -rf ~/.cache/huggingface/datasets/generator


# Intervention fit/load

In [6]:
import pickle

from src.config import root_folder
tokenizer_args=dict(padding="max_length", max_length=cfg.max_length, truncation=True, add_special_tokens=True)
            
def load_rep_reader(model, tokenizer, cfg, N_fit_examples=20, batch_size=2, rep_token = -1, n_difference = 1, direction_method = 'pca'):
    """
    We want one set of interventions per model
    
    So we always load a cached version if possible. to make it approx repeatable use the same dataset etc
    """
    model_name = cfg.model.replace('/', '-')
    intervention_f = root_folder / 'data' / 'interventions' / f'{model_name}.pkl'
    intervention_f.parent.mkdir(exist_ok=True, parents=True)
    if not intervention_f.exists():        
        
        hidden_layers = list(range(8, model.config.num_hidden_layers, 3))
        
        dataset_fit = load_preproc_dataset('imdb', cfg, tokenizer, N=N_fit_examples)
        
        rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)
        honesty_rep_reader = rep_reading_pipeline.get_directions(
            dataset_fit['question'], 
            rep_token=rep_token, 
            hidden_layers=hidden_layers, 
            n_difference=n_difference, 
            train_labels=dataset_fit['label_true'], 
            direction_method=direction_method,
            batch_size=batch_size,
            **tokenizer_args
        )
        # and save
        with open(intervention_f, 'wb') as f:
            pickle.dump(honesty_rep_reader, f)
            logger.info(f'Saved interventions to {intervention_f}')
    else:
        with open(intervention_f, 'rb') as f:
            honesty_rep_reader = pickle.load(f)
        logger.info(f'Loaded interventions from {intervention_f}')
            
    return honesty_rep_reader


In [7]:

# N_fit_examples = 20
N_fit_examples = 10
rep_token = -1

honesty_rep_reader = load_rep_reader(model, tokenizer, cfg, N_fit_examples=N_fit_examples, batch_size=batch_size, rep_token=rep_token)

hidden_layers = sorted(honesty_rep_reader.directions.keys())
hidden_layers


2023-10-25 13:32:42.237 | INFO     | __main__:load_rep_reader:39 - Loaded interventions from /media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/data/interventions/TheBloke-WizardCoder-Python-13B-V1.0-GPTQ.pkl
2023-10-25T13:32:42.237595+0800 INFO Loaded interventions from /media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/data/interventions/TheBloke-WizardCoder-Python-13B-V1.0-GPTQ.pkl


[8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38]

# Dataset

In [8]:
# load dataset
ds_name = 'imdb'
ds_tokens = load_preproc_dataset(ds_name, cfg, tokenizer)
ds_tokens


N_train_split = (len(ds_tokens) - N_fit_examples) //2

# split the dataset, it's preshuffled
dataset_fit = ds_tokens.select(range(N_fit_examples))
dataset_train = ds_tokens.select(range(N_fit_examples, N_train_split))
dataset_test = ds_tokens.select(range(N_train_split, len(ds_tokens)))
dataset_test


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# # fit
# # FIXME: load or save here if the inputs are the same....
# train_labels = dataset_fit['label_true']
# honesty_rep_reader = rep_reading_pipeline.get_directions(
#     dataset_fit['question'], 
#     rep_token=rep_token, 
#     hidden_layers=hidden_layers, 
#     n_difference=n_difference, 
#     train_labels=dataset_fit['label_true'], 
#     direction_method=direction_method,
#     batch_size=batch_size,
#     **tokenizer_args
# )
# honesty_rep_reader


In [ ]:
hidden_layers


In [ ]:
rep_reading_pipeline =  pipeline("rep-reading", model=model, tokenizer=tokenizer)


In [ ]:
# read direction for each example, layer
H_tests = rep_reading_pipeline(
    dataset_train['question'], 
    rep_token=rep_token, 
    hidden_layers=hidden_layers, 
    rep_reader=honesty_rep_reader,
    batch_size=batch_size, **tokenizer_args)

H_tests[0] # {Batch, layers}


# Control helpers

In [ ]:
inputs = dataset_train[:3]
inputs['question']


In [ ]:
def metrics(control_outputs_neg, baseline_outputs, control_outputs):
    signs = [-1, 0, 1]
    for i in range(len(baseline_outputs['ans'])):
        ranked = []
        for j, r in enumerate([control_outputs_neg, baseline_outputs, control_outputs]):        
            choices = r['answer_choices'][i]
            label = r['label_true'][i]
            ans = r['ans'][i].item()
            sign = signs[j]
            ranked.append(ans)
            choice_true = choices[label]
            if label==0:
                ans *= -1            
            print(f"==== Control ({signs[j]}) ====")
            print(f"Score: {ans:02.2%} of true ans `{choice_true}`")
            # print(f"Text ans: {r['text_ans'][i]}") 
        
        is_ranked = (np.argsort(ranked)==np.arange(3)).all()
        print(f"Ranked? {is_ranked} {ranked}")
        print()


# Control

In [ ]:


rep_control_pipeline2 = pipeline(
    "rep-control2", 
    model=model, 
    tokenizer=tokenizer, 
    layers=hidden_layers, 
    max_length=cfg.max_length,)
rep_control_pipeline2


In [ ]:

coeff=8.0
max_new_tokens=1

activations = {}
for layer in hidden_layers:
    activations[layer] = torch.tensor(coeff * honesty_rep_reader.directions[layer] * honesty_rep_reader.direction_signs[layer]).to(model.device).half()
    

activations_neg = {k:-v for k,v in activations.items()}


In [ ]:
# unit test: with multiple input types: single, list, generator, dataset
## single
input_types = {'single':dataset_train[0], 'list':dataset_train[:5], 'generator':iter(dataset_train.select(range(5))), 'dataset':dataset_train.select(range(5)).to_iterable_dataset()}
for name, ds in input_types.items():
    print(f"==== {name} ====")
    control_outputs = rep_control_pipeline2(ds, activations=activations, batch_size=2)
    r = list(control_outputs)
    print(len(r))



# control_outputs = rep_control_pipeline2(dataset_train[0], activations=activations)
# list(control_outputs)

# ## list
# control_outputs = rep_control_pipeline2(dataset_train[:5], activations=activations, batch_size=2)
# list(control_outputs)

# # generator
# ds = iter(dataset_train.select(range(5)))
# control_outputs = rep_control_pipeline2(ds, activations=activations, batch_size=2)
# list(control_outputs)

# # dataset
# ds = dataset_train.select(range(5)).to_iterable_dataset()
# control_outputs = rep_control_pipeline2(ds, activations=activations, batch_size=2)
# list(control_outputs)


In [ ]:


model.eval()
with torch.no_grad():
    baseline_outputs = rep_control_pipeline2(inputs, batch_size=batch_size)
    control_outputs = rep_control_pipeline2(inputs, activations=activations, batch_size=batch_size)
    control_outputs_neg = rep_control_pipeline2(inputs, activations=activations_neg, batch_size=batch_size)


metrics(control_outputs_neg, baseline_outputs, control_outputs)


In [ ]:
from torch.utils.data import Dataset


In [ ]:
import json
# from datasets import Dataset, DatasetInfo
import datasets
from src.config import root_folder
from pathvalidate import sanitize_filename
from src.helpers.ds import ds_keep_cols

def create_hs_ds(ds_name, ds_tokens, pipeline, activations=None, f = None, batch_size=2, split_type="train", debug=False):
    "create a dataset of hidden states."""
    
    N = len(ds_tokens)
    dataset_name = sanitize_filename(f"{cfg.model}_{ds_name}_{split_type}_{N}", replacement_text="_")
    f = str(root_folder / '.ds'/ f"{dataset_name}")
    
    info_kwargs = dict(extract_cfg=cfg.to_dict(), ds_name=ds_name, split_type=split_type, f=f, date=pd.Timestamp.now().isoformat(),)
    
    torch_cols = ['input_ids', 'attention_mask', 'choice_ids', 'question', 'answer_choices', 'example_i', 'label_true', 'sys_instr_name', 'template_name', 'instructed_to_lie']
    ds_t_subset = ds_keep_cols(ds_tokens, torch_cols)
    ds = ds_t_subset.to_iterable_dataset()
    # pipeline_it = rep_control_pipeline2(ds, batch_size=batch_size, **text_gen_kwargs)
    
    # first we make the calibration dataset with no intervention
    gen_kwargs = dict(
        model_inputs=ds,
        activations=activations,
        batch_size=batch_size,
    )
    
    if debug:
        # this allow us to debug in a single thread
        pipeline(**gen_kwargs)
    
    ds1 = datasets.Dataset.from_generator(
        generator=pipeline,
        info=datasets.DatasetInfo(
            description=json.dumps(info_kwargs, indent=2),
            config_name=f,
        ),
        gen_kwargs=gen_kwargs,
        num_proc=1,
    )
    logger.info(f"Created dataset {dataset_name} with {len(ds1)} examples at `{f}`")
    return ds1, f# TODO we need to save and cache for many split and datasets
rep_control_pipeline2



ds1, f = create_hs_ds('imdb', dataset_train, rep_control_pipeline2, split_type="train", debug=True)
ds1



In [ ]:
%debug


In [ ]:
dataset_train['answer_choices'][0]


In [ ]:
from torch.utils.data import Dataset
ds = dataset_train.to_iterable_dataset()
next(iter(ds))['answer_choices']


# To Datasets


In [ ]:
baseline_outputs
